In [18]:
import requests
import os 
import json
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
# Load apikey data

os.chdir('/Users/joemccann/MLProjects/alpaca_practice')
api_keys = json.load(open('apikey.json', 'r'))

In [3]:
def get_bar_data(api_keys, url, symbols, start='', end='', limit=1000, timeframe='1Day'):
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
        "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
    }
    
    params = {'symbols': symbols, 
          'timeframe':timeframe, 
          'start':start, 
          'end': end,
          'limit': limit}

    response = requests.get(url, headers=headers, params=params)

    data = response.json()

    df = pd.DataFrame(data['bars'][symbols])
    df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time',inplace=True)
    df = df[['open', 'close', 'high', 'low', 'trade_count', 'volume', 'volume_weighted_avg']]

    return df

def get_last_bar(api_keys, url, stocks):
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
        "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
    }

    list_of_dfs = []

    for stock in stocks:
        params = {'symbols': stock}
        
        response = requests.get(url, headers=headers, params=params)
        
        data = response.json()
    
        last_bar_df = pd.DataFrame(data['bars']).T
        last_bar_df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
        last_bar_df['stock_name'] = stock
        last_bar_df = last_bar_df[['stock_name'] + [col for col in last_bar_df.columns if col != 'stock_name']]
    
        list_of_dfs.append(last_bar_df)
    
    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

In [ ]:
# Get Bar Data

In [12]:
url = "https://data.alpaca.markets/v2/stocks/bars?"
symbols = 'TSLA'
start = '2020-01-10'
end =  '2024-01-10'

bar_data_df = get_bar_data(api_keys, url, symbols, start, end)

In [13]:
df.head()

,open,close,high,low,trade_count,volume,volume_weighted_avg
time,,,,,,,
2020-01-10 05:00:00+00:00,481.790,478.15,484.94,473.7000,181667,13217075,478.957386
2020-01-13 05:00:00+00:00,493.500,524.86,525.63,492.0000,369915,26916914,510.914630
2020-01-14 05:00:00+00:00,544.255,537.92,547.41,524.9000,406428,29600289,538.243513
2020-01-15 05:00:00+00:00,529.760,518.50,537.84,516.7853,261961,17643746,529.099215
2020-01-16 05:00:00+00:00,493.750,513.49,514.46,492.1700,325060,22123951,504.239601


In [14]:
#Last Trade

In [15]:
last_url = "https://data.alpaca.markets/v2/stocks/bars/latest?"
stocks = ['AMZN', 'TSLA', 'AAPL']

last_trade_df = get_last_bar(api_keys, last_url, stocks)

In [16]:
last_trade_df

,stock_name,close,high,low,trade_count,open,time,volume,volume_weighted_avg
0,AMZN,154.6,154.645,154.46,187,154.59,2024-01-12T20:59:00Z,19449,154.558886
1,TSLA,217.98,217.98,217.98,1,217.98,2024-01-12T21:21:00Z,100,217.98
2,AAPL,185.83,185.975,185.72,260,185.96,2024-01-12T20:59:00Z,22286,185.82078
